In [ ]:
import numpy as np
import os
import sys
import pickle as pkl
import pandas as pd

In [ ]:
sys.path.append('src')
from datasets import dataset_names

In [ ]:
datasets = dataset_names
print(datasets)

In [ ]:
data_dir = 'data'

In [ ]:
algo_names = ['LR', 'LR-CV', 'TP', 'TP-CV', 'TP-NW']

In [ ]:
metrics = ['Precision@3', 'Precision@5', 'Precision@K', 'RankingLoss', 'F1']

In [ ]:
indices = pd.MultiIndex.from_product([dataset_names, algo_names], names=['Dataset', 'Method'])

In [ ]:
#df_train = pd.DataFrame(index=indices, columns=metrics)
df_test  = pd.DataFrame(index=indices, columns=metrics)

In [ ]:
fprec_tp0 = os.path.join(data_dir, 'perf-tp-base.pkl')
fprec_tp1 = os.path.join(data_dir, 'perf-tp-prec.pkl')
fprec_tp2 = os.path.join(data_dir, 'perf-tp-noclsw.pkl')
fprec_lr0 = os.path.join(data_dir, 'perf-lr-base.pkl')
fprec_lr1 = os.path.join(data_dir, 'perf-lr-prec.pkl')
records_tp0 = pkl.load(open(fprec_tp0, 'rb'))
records_tp1 = pkl.load(open(fprec_tp1, 'rb'))
records_tp2 = pkl.load(open(fprec_tp2, 'rb'))
records_lr0 = pkl.load(open(fprec_lr0, 'rb'))
records_lr1 = pkl.load(open(fprec_lr1, 'rb'))

In [ ]:
#records_lr1

In [ ]:
def f1_str(records_dict, ds, split):
    return ', '.join(['$%.4f$' % v[0] for v in records_dict[ds][split]['F1']])

for metric in metrics:
    for ds in datasets:
        if metric == 'F1':
            df_test.loc[(ds, 'TP'), metric] = f1_str(records_tp0, ds, 'Test')
            df_test.loc[(ds, 'TP-CV'), metric] = f1_str(records_tp1, ds, 'Test')
            df_test.loc[(ds, 'TP-NW'), metric] = f1_str(records_tp2, ds, 'Test')
        else:
            df_test.loc[(ds, 'TP'), metric] = records_tp0[ds]['Test' ][metric][0]
            df_test.loc[(ds, 'TP-CV'), metric] = records_tp1[ds]['Test' ][metric][0]
            df_test.loc[(ds, 'TP-NW'), metric] = records_tp2[ds]['Test' ][metric][0]
        
        df_test.loc[(ds, 'LR'), metric] = records_lr0[ds]['Test' ][metric][0]
        df_test.loc[(ds, 'LR-CV'), metric] = records_lr1[ds]['Test' ][metric][0]

In [ ]:
def highlight_max(s):
    is_max = s == s.max()
    return ['\\textbf{%.4f}' % v if v else '' for v in is_max]

In [ ]:
tab_test = df_test.to_latex(float_format=lambda x: '$%.4f$' % x, na_rep='-', multirow=True, escape=False)

In [ ]:
print('\\begin{table}[!h]')
print('\centering')
print('\\caption{Performance on test set}')
print('\\label{tab:perf}')    
print(tab_test)
print('\\end{table}')

In [ ]:
datasets = ['yeast', 'scene', 'bibtex', 'bookmarks']

In [ ]:
#split = 'Train'
split = 'Test'

In [ ]:
# load perf records
perf_files = [os.path.join(data_dir, 'perf-' + algo + '.pkl') for algo in algos]
perf_dicts = [pkl.load(open(fname, 'rb')) for fname in perf_files]

def perfstr(perftuple):
    return '$%.4f \\pm  %.3f$' % (perftuple[0], perftuple[1])

for metric in metrics:
    tabstr = []
    tabstr.append('\\begin{table}[!h]')
    tabstr.append('\centering')
    tabstr.append('\\caption{Performance in terms of average %s (%s)}' % (metric, split))
    tabstr.append('\\begin{tabular}{lcccc} \\\\ \\hline \\hline')
    tabstr.append(' & ' + ' & '.join([ds for ds in datasets]) + ' \\\\')
    for j in range(len(algos)):
        tabstr.append(algo_names[j] + ' & '
                      + ' & '.join([perfstr(perf_dicts[j][ds][split][metric]) \
                                    if ds in perf_dicts[j] else '-' for ds in datasets ])
                      + ' \\\\')
    tabstr.append('\\hline')
    tabstr.append('\\end{tabular}')
    tabstr.append('\\end{table}')
    print('\n'.join(tabstr))
    print()